In [1]:
import gc
import json
import numpy as np 
import pandas as pd
import re

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
from sklearn.metrics import accuracy_score, f1_score
from tqdm import tqdm_notebook as tqdm
import Levenshtein 

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction import text
from sklearn.model_selection import GroupKFold
from sklearn.linear_model import LogisticRegression

from scipy import spatial
import lightgbm as lgb
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler

/kaggle/input/tensorflow2-question-answering/simplified-nq-train.jsonl
/kaggle/input/tensorflow2-question-answering/simplified-nq-test.jsonl
/kaggle/input/tensorflow2-question-answering/sample_submission.csv


Using TensorFlow backend.


In [2]:
html_tags = ['<P>', '</P>', '<Table>', '</Table>', '<Tr>', '</Tr>', '<Ul>', '<Ol>', '<Dl>', '</Ul>', '</Ol>', \
             '</Dl>', '<Li>', '<Dd>', '<Dt>', '</Li>', '</Dd>', '</Dt>']
r_buf = ['is', 'are', 'do', 'does', 'did', 'was', 'were', 'will', 'can', 'the', 'a', 'an', 'of', 'in', 'and', 'on', \
         'what', 'where', 'when', 'which'] + html_tags

def clean(x):
    x = x.lower()
    for r in r_buf:
        x = x.replace(r, '')
    x = re.sub(' +', ' ', x)
    return x

## Prepairing train dataset

Here we are going to collect data from json files and format it to the tabular data. We will formulate the problem as a binary classification problem and will try to classify if chosen candidate is an answer.

In [3]:
%%time
n_samples = 50000 # Number of samples to read from the train.json

# Read data from train.json and prepare features
ids = []
question_tfidfs = []
answer_tfidfs = []
candidates_str = []
targets = []
targets_str = []
targets_str_short = []
features = []
rank_features = []

with open('/kaggle/input/tensorflow2-question-answering/simplified-nq-train.jsonl', 'r') as json_file:
    cnt = 0
    for line in tqdm(json_file):
        json_data = json.loads(line) 
        
        # TFIDF for document
        stop_words = text.ENGLISH_STOP_WORDS.union(["book"])
        tfidf = TfidfVectorizer(ngram_range=(1, 1), stop_words=stop_words)
        tfidf.fit([json_data['document_text']])

        # TFIDF for question
        question = json_data['question_text']
        question_tfidf = tfidf.transform([question]).todense()
        
        # Collect annotations
        start_token_true = json_data['annotations'][0]['long_answer']['start_token']
        end_token_true = json_data['annotations'][0]['long_answer']['end_token']
        
        # Collect short annotations
        if json_data['annotations'][0]['yes_no_answer'] == 'NONE':
            if len(json_data['annotations'][0]['short_answers']) > 0:
                s_ans = str(json_data['annotations'][0]['short_answers'][0]['start_token']) + ':' + \
                    str(json_data['annotations'][0]['short_answers'][0]['end_token'])
            else:
                s_ans = ''
        else:
            s_ans = json_data['annotations'][0]['yes_no_answer']

        cos_d_buf = []
        euc_d_buf = []
        lev_d_buf = []
        
        doc_tokenized = json_data['document_text'].split(' ')
        candidates = json_data['long_answer_candidates']
        candidates = [c for c in candidates if c['top_level'] == True]
        
        if start_token_true != -1:
            for c in candidates:
                ids.append(str(json_data['example_id']))

                # TFIDF for candidate answer
                start_token = c['start_token']
                end_token = c['end_token']
                answer = ' '.join(doc_tokenized[start_token:end_token])
                answer_tfidf = tfidf.transform([answer]).todense()

                # Extract some features
                cos_d = spatial.distance.cosine(question_tfidf, answer_tfidf)
                euc_d = np.linalg.norm(question_tfidf - answer_tfidf)
                lev_d = Levenshtein.distance(clean(question), clean(answer))
                lev_r = Levenshtein.ratio(clean(question), clean(answer))
                jar_s = Levenshtein.jaro(clean(question), clean(answer))
                jaw_s = Levenshtein.jaro_winkler(clean(question), clean(answer))
                tfidf_score = np.sum(question_tfidf*answer_tfidf.T)
                question_tfidf_sum = np.sum(question_tfidf)
                answer_tfidf_sum = np.sum(answer_tfidf)

                features.append([
                    cos_d, 
                    euc_d, 
                    lev_d, 
                    lev_r, 
                    jar_s, 
                    jaw_s, 
                    tfidf_score, 
                    question_tfidf_sum, 
                    answer_tfidf_sum
                ])
                
                cos_d_buf.append(cos_d)
                euc_d_buf.append(euc_d)
                lev_d_buf.append(lev_d)

                targets_str.append(str(start_token_true) + ':' + str(end_token_true))
                candidates_str.append(str(start_token) + ':' + str(end_token))
                targets_str_short.append(s_ans)

                # Get target
                if start_token == start_token_true and end_token == end_token_true:
                    target = 1
                else:
                    target = 0
                targets.append(target)

            rank_cos_d = np.argsort(cos_d_buf)
            rank_euc_d = np.argsort(euc_d_buf)
            rank_lev_d = np.argsort(lev_d_buf)
            rank_cos_d_ismin = (cos_d_buf == np.nanmin(cos_d_buf)).astype(int)
            rank_euc_d_ismin = (euc_d_buf == np.nanmin(euc_d_buf)).astype(int)
            rank_lev_d_ismin = (lev_d_buf == np.nanmin(lev_d_buf)).astype(int)
            rank_features.append(np.array([rank_cos_d, rank_euc_d, rank_lev_d, \
                                           rank_cos_d_ismin, rank_euc_d_ismin, rank_lev_d_ismin]).T)

        cnt += 1
        if cnt >= n_samples:
            break
        
train = pd.DataFrame()
train['example_id'] = ids
train['target'] = targets
train['CorrectString'] = targets_str
train['CorrectString_short'] = targets_str_short
train['CandidateString'] = candidates_str

features = np.array(features)
features_df = pd.DataFrame(features)
features_df.columns = [f'feature_{i}' for i in range(features.shape[1])]
train = pd.concat([train, features_df], axis=1)

rank_features = np.concatenate(rank_features, axis=0)
rank_features_df = pd.DataFrame(rank_features)
rank_features_df.columns = [f'rank_feature_{i}' for i in range(rank_features.shape[1])]
train = pd.concat([train, rank_features_df], axis=1)

del features, features_df, \
    rank_features, rank_features_df
gc.collect()

train.to_csv('train_data.csv', index=False)
print(f'train.shape: {train.shape}')
print(f'Mean target: {train.target.mean()}')
train.head(20)

/opt/conda/lib/python3.6/site-packages/scipy/spatial/distance.py:702: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:101: RuntimeWarning: All-NaN axis encountered


train.shape: (964518, 20)
Mean target: 0.02441737738435156
CPU times: user 43min 30s, sys: 17.1 s, total: 43min 47s
Wall time: 41min 11s


,example_id,target,CorrectString,CorrectString_short,CandidateString,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,rank_feature_0,rank_feature_1,rank_feature_2,rank_feature_3,rank_feature_4,rank_feature_5
0,5655493461695504401,0,1952:2019,1960:1969,14:170,1.000000,1.414214,717.0,0.072072,0.318764,0.318764,0.000000,2.236068,4.557991,11,11,12,0,0,0
1,5655493461695504401,0,1952:2019,1960:1969,170:321,0.936099,1.368283,818.0,0.065984,0.339630,0.339630,0.063901,2.236068,2.898563,16,16,15,0,0,0
2,5655493461695504401,0,1952:2019,1960:1969,321:460,0.889718,1.333955,633.0,0.083573,0.435696,0.435696,0.110282,2.236068,6.247162,17,17,5,0,0,0
3,5655493461695504401,0,1952:2019,1960:1969,563:746,0.833794,1.291352,828.0,0.067416,0.391103,0.391103,0.166206,2.236068,7.209953,13,13,11,0,0,0
4,5655493461695504401,0,1952:2019,1960:1969,746:852,0.832295,1.290190,483.0,0.106618,0.394497,0.394497,0.167705,2.236068,6.250000,18,18,17,0,0,0
5,5655493461695504401,0,1952:2019,1960:1969,855:869,0.817426,1.278613,50.0,0.346939,0.515287,0.515287,0.182574,2.236068,2.449490,23,23,20,0,0,0
6,5655493461695504401,0,1952:2019,1960:1969,873:970,1.000000,1.414214,464.0,0.114068,0.425830,0.425830,0.000000,2.236068,5.212860,12,12,22,0,0,0
7,5655493461695504401,0,1952:2019,1960:1969,970:1044,1.000000,1.414214,317.0,0.143617,0.424727,0.424727,0.000000,2.236068,5.125693,5,5,24,0,0,0
8,5655493461695504401,0,1952:2019,1960:1969,1044:1107,0.896720,1.339194,308.0,0.157182,0.403116,0.403116,0.103280,2.236068,4.503332,4,4,27,0,0,0
9,5655493461695504401,0,1952:2019,1960:1969,1111:1175,0.882556,1.328575,270.0,0.180723,0.397222,0.397222,0.117444,2.236068,4.727032,3,3,19,0,0,0


## Preparing test dataset

In [4]:
%%time
ids = []
question_tfidfs = []
answer_tfidfs = []
candidates_str = []
targets = []
targets_str = []
features = []
rank_features = []

with open('/kaggle/input/tensorflow2-question-answering/simplified-nq-test.jsonl', 'r') as json_file:
    for line in tqdm(json_file):
        json_data = json.loads(line) 
        
        # TFIDF for document
        stop_words = text.ENGLISH_STOP_WORDS.union(["book"])
        tfidf = TfidfVectorizer(ngram_range=(1, 1), stop_words=stop_words)
        tfidf.fit([json_data['document_text']])
        
        # TFIDF for question
        question = json_data['question_text']
        question_tfidf = tfidf.transform([question]).todense()
        
        doc_tokenized = json_data['document_text'].split(' ')
        candidates = json_data['long_answer_candidates']
        candidates = [c for c in candidates if c['top_level'] == True]
        
        cos_d_buf = []
        euc_d_buf = []
        lev_d_buf = []
        
        for c in candidates:
            ids.append(str(json_data['example_id']))
            
            # TFIDF for candidate answer
            start_token = c['start_token']
            end_token = c['end_token']
            answer = ' '.join(doc_tokenized[start_token:end_token])
            answer_tfidf = tfidf.transform([answer]).todense()
            
            # Extract some features
            cos_d = spatial.distance.cosine(question_tfidf, answer_tfidf)
            euc_d = np.linalg.norm(question_tfidf - answer_tfidf)
            lev_d = Levenshtein.distance(clean(question), clean(answer))
            lev_r = Levenshtein.ratio(clean(question), clean(answer))
            jar_s = Levenshtein.jaro(clean(question), clean(answer))
            jaw_s = Levenshtein.jaro_winkler(clean(question), clean(answer))
            tfidf_score = np.sum(question_tfidf*answer_tfidf.T)
            question_tfidf_sum = np.sum(question_tfidf)
            answer_tfidf_sum = np.sum(answer_tfidf)

            features.append([
                cos_d, 
                euc_d, 
                lev_d, 
                lev_r, 
                jar_s, 
                jaw_s, 
                tfidf_score, 
                question_tfidf_sum, 
                answer_tfidf_sum
            ])

            cos_d_buf.append(cos_d)
            euc_d_buf.append(euc_d)
            lev_d_buf.append(lev_d)
            
            candidates_str.append(str(start_token) + ':' + str(end_token))
        
        rank_cos_d = np.argsort(cos_d_buf)
        rank_euc_d = np.argsort(euc_d_buf)
        rank_lev_d = np.argsort(lev_d_buf)
        rank_cos_d_ismin = (cos_d_buf == np.nanmin(cos_d_buf)).astype(int)
        rank_euc_d_ismin = (euc_d_buf == np.nanmin(euc_d_buf)).astype(int)
        rank_lev_d_ismin = (lev_d_buf == np.nanmin(lev_d_buf)).astype(int)
        rank_features.append(np.array([rank_cos_d, rank_euc_d, rank_lev_d, \
                                       rank_cos_d_ismin, rank_euc_d_ismin, rank_lev_d_ismin]).T)
        
test = pd.DataFrame()
test['example_id'] = ids
test['CandidateString'] = candidates_str

features = np.array(features)
features_df = pd.DataFrame(features)
features_df.columns = [f'feature_{i}' for i in range(features.shape[1])]
test = pd.concat([test, features_df], axis=1)

rank_features = np.concatenate(rank_features, axis=0)
rank_features_df = pd.DataFrame(rank_features)
rank_features_df.columns = [f'rank_feature_{i}' for i in range(rank_features.shape[1])]
test = pd.concat([test, rank_features_df], axis=1)

del features, features_df, rank_features, rank_features_df
gc.collect()

test.to_csv('test_data.csv', index=False)
print(f'test.shape: {test.shape}')
test.head(10)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:72: RuntimeWarning: All-NaN axis encountered



test.shape: (13412, 17)
CPU times: user 28.7 s, sys: 168 ms, total: 28.8 s
Wall time: 28.2 s


,example_id,CandidateString,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,rank_feature_0,rank_feature_1,rank_feature_2,rank_feature_3,rank_feature_4,rank_feature_5
0,-1220107454853145579,18:136,0.850419,1.304162,477.0,0.104673,0.425633,0.425633,0.149581,2.236068,3.631420,1,1,1,0,0,0
1,-1220107454853145579,141:211,0.427402,0.924556,222.0,0.206406,0.496494,0.496494,0.572598,2.236068,4.225217,5,5,4,1,1,1
2,-1220107454853145579,240:336,0.836092,1.293130,376.0,0.133333,0.448399,0.448399,0.163908,2.236068,5.986303,3,3,5,0,0,0
3,-1220107454853145579,336:425,0.727202,1.205987,321.0,0.142857,0.441379,0.441379,0.272798,2.236068,5.337450,2,2,3,0,0,0
4,-1220107454853145579,425:488,0.841886,1.297602,230.0,0.188153,0.431496,0.431496,0.158114,2.236068,4.949747,4,4,2,0,0,0
5,-1220107454853145579,488:570,0.660450,1.149304,266.0,0.167183,0.443715,0.443715,0.339550,2.236068,4.664005,0,0,0,0,0,0
6,8777415633185303067,23:190,0.868018,1.317587,873.0,0.056216,0.388144,0.388144,0.131982,1.732051,3.352803,8,8,13,0,0,0
7,8777415633185303067,190:269,0.505128,1.005115,312.0,0.137741,0.424499,0.424499,0.494872,1.732051,5.571429,1,1,7,0,0,0
8,8777415633185303067,269:363,0.617640,1.111432,380.0,0.120370,0.460943,0.460943,0.382360,1.732051,6.490209,14,14,1,0,0,0
9,8777415633185303067,363:507,0.612899,1.107158,559.0,0.081967,0.438645,0.438645,0.387101,1.732051,6.034306,3,3,5,0,0,0


## Build the model

In [5]:
p_buf = []
n_splits = 4

kf = GroupKFold(
    n_splits=n_splits)

err_buf = []   

cols_to_drop = ['example_id', 'target', 'CorrectString', 'CorrectString_short', 'CandidateString']

X = train.drop(cols_to_drop, axis=1, errors='ignore')
y = train['target'].values
g = train['example_id'].values

X_test = test.drop(cols_to_drop, axis=1, errors='ignore')
id_test = test['example_id'].values

print(f'X.shape: {X.shape}, y.shape: {y.shape}')
print(f'X_test.shape: {X_test.shape}')

n_features = X.shape[1]

lgb_params = {
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': 'binary_logloss',
    'max_depth': 16,
    'learning_rate': 0.005, 
    'feature_fraction': 0.9,
    'bagging_fraction': 0.9,
    'bagging_freq': 5,
    'verbose': -1,
    'num_threads': 5,
}

for fold_i, (train_index, valid_index) in enumerate(kf.split(X, y, g)):
    print('Fold {}/{}'.format(fold_i + 1, n_splits))
    params = lgb_params.copy() 
    
    X_train, y_train = X.iloc[train_index], y[train_index]
    X_valid, y_valid = X.iloc[valid_index], y[valid_index]

    print(f'X_train.shape: {X_train.shape}, X_valid.shape: {X_valid.shape}')
    feature_names = list(X_train.columns)

    lgb_train = lgb.Dataset(
        X_train, 
        y_train, 
        feature_name=feature_names,
        )
    lgb_train.raw_data = None

    lgb_valid = lgb.Dataset(
        X_valid, 
        y_valid,
        feature_name=feature_names,
        )
    lgb_valid.raw_data = None

    model = lgb.train(
        params,
        lgb_train,
        num_boost_round=1000,
        valid_sets=[lgb_train, lgb_valid],
        early_stopping_rounds=100, 
        verbose_eval=100, 
    )

    # Feature importance
    if fold_i == 0:
        importance = model.feature_importance()
        model_fnames = model.feature_name()
        tuples = sorted(zip(model_fnames, importance), key=lambda x: x[1])[::-1]
        tuples = [x for x in tuples if x[1] > 0]
        print('Important features:')
        for i in range(20):
            if i < len(tuples):
                print(i, tuples[i])
            else:
                break

    # Evaluate model
    p = model.predict(X.loc[valid_index], num_iteration=model.best_iteration)
    valid_df = train.loc[valid_index]
    valid_df['pred'] = p
    pred_df = valid_df.sort_values('pred', ascending=True).groupby('example_id').tail(1)

    pred_df_long = pred_df[['example_id', 'CorrectString', 'CandidateString']]
    pred_df_long.rename({'CandidateString': 'PredictionString'}, axis=1, inplace=True)
    pred_df_long['example_id'] = pred_df_long['example_id'].apply(lambda x: x + '_long')

    pred_df_short = pred_df[['example_id', 'CorrectString_short', 'CandidateString']]
    pred_df_short.rename({'CorrectString_short': 'CorrectString', 'CandidateString': 'PredictionString'}, \
                         axis=1, inplace=True)
    pred_df_short['example_id'] = pred_df_short['example_id'].apply(lambda x: x + '_short')
    pred_df_short['PredictionString'] = ''

    pred_df = pd.concat([pred_df_long, pred_df_short], axis=0).sort_values('example_id')
#     print(pred_df.head(20))

    err = f1_score(pred_df['CorrectString'].values, pred_df['PredictionString'].values, average='micro')
    print('{} F1: {}'.format(fold_i, err))
    
    # Inference on test data
    p_test = model.predict(X_test[feature_names], num_iteration=model.best_iteration)
    p_buf.append(p_test)
    err_buf.append(err)

#     if fold_i >= 0: # Comment this to run several folds
#         break

    del model, lgb_train, lgb_valid, p
    gc.collect()

X.shape: (964518, 15), y.shape: (964518,)
X_test.shape: (13412, 15)
Fold 1/4
X_train.shape: (723388, 15), X_valid.shape: (241130, 15)
Training until validation scores don't improve for 100 rounds
[100]	training's binary_logloss: 0.0993161	valid_1's binary_logloss: 0.0993687
[200]	training's binary_logloss: 0.0945578	valid_1's binary_logloss: 0.0948359
[300]	training's binary_logloss: 0.0921776	valid_1's binary_logloss: 0.0926588
[400]	training's binary_logloss: 0.0908165	valid_1's binary_logloss: 0.0914621
[500]	training's binary_logloss: 0.0899019	valid_1's binary_logloss: 0.0907145
[600]	training's binary_logloss: 0.0892522	valid_1's binary_logloss: 0.0902192
[700]	training's binary_logloss: 0.0887774	valid_1's binary_logloss: 0.0898959
[800]	training's binary_logloss: 0.0883996	valid_1's binary_logloss: 0.0896857
[900]	training's binary_logloss: 0.0880731	valid_1's binary_logloss: 0.0895352
[1000]	training's binary_logloss: 0.0877794	valid_1's binary_logloss: 0.0894339
Did not meet 

/opt/conda/lib/python3.6/site-packages/pandas/core/frame.py:4238: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(**kwargs)
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:90: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:95: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.ht

0 F1: 0.2640521823159929
Fold 2/4
X_train.shape: (723388, 15), X_valid.shape: (241130, 15)
Training until validation scores don't improve for 100 rounds
[100]	training's binary_logloss: 0.0993091	valid_1's binary_logloss: 0.0995176
[200]	training's binary_logloss: 0.0945403	valid_1's binary_logloss: 0.0948462
[300]	training's binary_logloss: 0.0922111	valid_1's binary_logloss: 0.0926232
[400]	training's binary_logloss: 0.0908269	valid_1's binary_logloss: 0.0913615
[500]	training's binary_logloss: 0.0898947	valid_1's binary_logloss: 0.0905702
[600]	training's binary_logloss: 0.0892187	valid_1's binary_logloss: 0.0900297
[700]	training's binary_logloss: 0.0887479	valid_1's binary_logloss: 0.089699
[800]	training's binary_logloss: 0.0883744	valid_1's binary_logloss: 0.0894728
[900]	training's binary_logloss: 0.0880617	valid_1's binary_logloss: 0.0893327
[1000]	training's binary_logloss: 0.0877846	valid_1's binary_logloss: 0.0892367
Did not meet early stopping. Best iteration is:
[1000]	tr

In [6]:
err_mean = np.mean(err_buf)
err_std = np.std(err_buf)
print('F1 = {:.4f} +/- {:.4f}'.format(err_mean, err_std))

F1 = 0.2636 +/- 0.0013


## Prepare submission

In [7]:
valid_df = train.loc[valid_index]
test['pred'] = np.mean(p_buf, axis=0)
pred_df = test.sort_values('pred', ascending=True).groupby('example_id').tail(1)

pred_df_long = pred_df[['example_id', 'CandidateString']]
pred_df_long.rename({'CandidateString': 'PredictionString'}, axis=1, inplace=True)
pred_df_long['example_id'] = pred_df_long['example_id'].apply(lambda x: str(x) + '_long')

pred_df_short = pred_df[['example_id', 'CandidateString']]
pred_df_short.rename({'CandidateString': 'PredictionString'}, axis=1, inplace=True)
pred_df_short['example_id'] = pred_df_short['example_id'].apply(lambda x: str(x) + '_short')
pred_df_short['PredictionString'] = ''

subm = pd.concat([pred_df_long, pred_df_short], axis=0).sort_values('example_id')
subm.to_csv('submission.csv', index=False)
print(f'subm.shape: {subm.shape}')
print(subm.head(20))

subm.shape: (692, 2)
                       example_id PredictionString
10999   -1011141123527297803_long          542:582
10999  -1011141123527297803_short                 
5036    -1028916936938579349_long          781:923
5036   -1028916936938579349_short                 
9426    -1055197305756217938_long          221:335
9426   -1055197305756217938_short                 
5467    -1074129516932871805_long        3491:3607
5467   -1074129516932871805_short                 
2915    -1114334749483663139_long         744:3809
2915   -1114334749483663139_short                 
1368    -1152268629614456016_long          491:526
1368   -1152268629614456016_short                 
1871    -1219507076732106786_long        2398:2500
1871   -1219507076732106786_short                 
1       -1220107454853145579_long          141:211
1      -1220107454853145579_short                 
10661   -1237358188352001279_long          815:857
10661  -1237358188352001279_short                 
2261    -1

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cave